### Run this in Python kernel

To reset font style of output (in case it is not the default one)

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure.test :refer :all]
            [oz.notebook.clojupyter :as oz]
            [clojure-backtesting.data :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.large-data :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure.string :as str]
            [clojure.pprint :as pprint]
            [java-time :as t]
            [clojupyter.kernel.version :as ver]
            [clojupyter.misc.helper :as helper]
  ) ;; require all libriaries from core
  (:use clojure.pprint)
)

nil

### Import dataset
You could either load the extract dataset (with 3-4 tickers), or load the large dataset (around 15 GB in size). Note that the large dataset would take a bit of time to run different strategies.

#### Run with the extract dataset

In [2]:
; path to dataset = "../resources/CRSP-extract.csv"
; change it to the relative to your own dataset
;
(reset! data-set (add-aprc (read-csv-row "../resources/CRSP-extract.csv")));

#### Run with the large dataset

In [3]:
(load-large-dataset "../resources/CRSP-extract.csv" "main" add-aprc)
(set-main "main")

true

### Initialise portfolio

In [4]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-16" 10000);

In [5]:
(keys (deref available-tics))
;; (get-line)

("AAPL")

### Write a strategy

The following code implements a simple trading strategy:

In a timespan of 10 days (inclusive of today),
- Buy 50 stocks of AAPL on the first day
- Sell 10 stocks of AAPL on every other day

### (1) Running the strategy in non-lazy mode
- Note that you should have loaded the **extract** dataset

In [6]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days) ;; check if num-of-days is > 0
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days) ;; check if num-of-days == 10
            (do
                (order "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get-date)))
            )
        )
        (if (odd? @num-of-days) ;; check if num-of-days is odd
            (do
                (order "AAPL" -10) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get-date)))
            )
        )
        
        (update-eval-report (get-date)) ;; update the evaluation metrics every day
        (println (get-date))
        ; move on to the next trading day
        (next-date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

; check whether counter == 0
(println ((fn [counter] (str "Counter: " counter)) @num-of-days))

Buy 50 stocks of AAPL on 1980-12-16
1980-12-16
Sell 10 stocks of AAPL on 1980-12-17
1980-12-17
1980-12-18
Sell 10 stocks of AAPL on 1980-12-19
1980-12-19
1980-12-22
Sell 10 stocks of AAPL on 1980-12-23
1980-12-23
1980-12-24
Sell 10 stocks of AAPL on 1980-12-26
1980-12-26
1980-12-29
Sell 10 stocks of AAPL on 1980-12-30
1980-12-30
Counter: 0


nil

### (2) Running the strategy in lazy mode
- Note that you should have loaded the **large** dataset

In [14]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days) ;; check if num-of-days is > 0
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days) ;; check if num-of-days == 10
            (do
                (order-lazy "AAPL" 50) ; buy 50 stocks
                (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get-date)))
            )
        )
        (if (odd? @num-of-days) ;; check if num-of-days is odd
            (do
                (order-lazy "AAPL" -10) ; sell 10 stocks
                (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get-date)))
            )
        )
        
        (update-eval-report (get-date)) ;; update the evaluation metrics every day
        
        ; move on to the next trading day
        (next-date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

; check whether counter == 0
(println ((fn [counter] (str "Counter: " counter)) @num-of-days))

Buy 50 stocks of AAPL on 1980-12-16
Sell 10 stocks of AAPL on 1980-12-17
Sell 10 stocks of AAPL on 1980-12-19
Sell 10 stocks of AAPL on 1980-12-23
Sell 10 stocks of AAPL on 1980-12-26
Sell 10 stocks of AAPL on 1980-12-30
Counter: 0


nil

#### The following functions could be run in both lazy and non-lazy modes.

### Check order record

In [5]:
(pprint/print-table (deref order-record))


|      :date | :tic |  :price | :aprc | :quantity |
|------------+------+---------+-------+-----------|
| 1980-12-17 | AAPL | 25.9375 | 26.09 |        50 |
| 1980-12-17 | AAPL | 25.9375 | 26.09 |       -50 |


nil

### Check portfolio record

In [8]:
;; view final portfolio
(view-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A | 10119.51 |


nil

In [9]:
;; view portfolio value and return
(view-portfolio-record -1)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-16 |  $10000.00 |      0.00% |    0.00% | $0.00 |      0.00 |
| 1980-12-17 |  $10000.00 |      0.00% |    0.00% | $0.00 |      0.00 |
| 1980-12-18 |  $10016.25 |      0.00% |    0.07% | $0.00 |      0.00 |
| 1980-12-19 |  $10043.73 |      0.12% |    0.19% | $0.00 |      0.00 |
| 1980-12-22 |  $10066.29 |      0.00% |    0.29% | $0.00 |      0.00 |
| 1980-12-23 |  $10081.29 |      0.06% |    0.35% | $0.00 |      0.00 |
| 1980-12-24 |  $10100.29 |      0.00% |    0.43% | $0.00 |      0.00 |
| 1980-12-26 |  $10122.77 |      0.10% |    0.53% | $0.00 |      0.00 |
| 1980-12-29 |  $10126.41 |      0.00% |    0.55% | $0.00 |      0.00 |
| 1980-12-30 |  $10123.21 |     -0.01% |    0.53% | $0.00 |      0.00 |
| 1980-12-31 |  $10119.51 |     -0.02% |    0.52% | $0.00 |      0.00 |


nil

### Generate evaluation report

In [10]:
(eval-report -1)


|      :date | :tot-value |    :vol |  :r-vol |  :sharpe | :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+----------+-----------+---------+---------------|
| 1980-12-16 |     $10000 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $0 |        0.0000 |
| 1980-12-17 |     $10016 | 0.0407% | 0.0407% |  1.7321% |   1.7321% |      $8 |      100.0000 |
| 1980-12-18 |     $10016 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $8 |        0.0000 |
| 1980-12-19 |     $10066 | 0.0598% | 0.0598% |  4.8019% |   4.8019% |     $22 |      100.0000 |
| 1980-12-22 |     $10066 | 0.0532% | 0.0532% |  5.3929% |   5.3929% |     $22 |      100.0000 |
| 1980-12-23 |     $10100 | 0.0499% | 0.0499% |  8.6792% |   8.6792% |     $25 |      100.0000 |
| 1980-12-24 |     $10100 | 0.0475% | 0.0475% |  9.1298% |   9.1298% |     $25 |      100.0000 |
| 1980-12-26 |     $10126 | 0.0477% | 0.0477% | 11.4442% |  11.4442% |     $25 |      100.0000 |
| 1980-12-29 |     $10126 | 0

nil

In [12]:
(defn current-stack-trace []
      (.getStackTrace (Thread/currentThread)))

(defn is-repl-stack-element [stack-element]
      (and (= "clojure.main$repl" (.getClassName  stack-element))
           (= "doInvoke"          (.getMethodName stack-element))))

(defn is-in-repl []
      (some is-repl-stack-element (current-stack-trace)))

#'clojure-backtesting.demo/is-in-repl

In [13]:
(is-in-repl)

true

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio daily return

In [7]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [8]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "port-value")
  data))

#'clojure-backtesting.demo/data-to-plot

In [9]:
(first data-to-plot)

{:date "1980-12-16", :tot-value 10000.0, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :margin 0.0, :plot "port-value"}

In [10]:
(plot data-to-plot :plot :date :daily-ret false)

[I 07:26:07.058 Clojupyter] oz.core:273 -- Starting up server on port 10666
[E 07:26:09.473 Clojupyter] oz.core:1147 -- error sending plot to server: java.net.BindException: Address already in use
[E 07:26:09.475 Clojupyter] oz.core:1148 -- Try using a different port?


nil

### 2. Plot volatility

In [15]:
(def data (deref eval-record))

#'clojure-backtesting.demo/data

In [16]:
(first data)

{:date "1980-12-16", :tot-value 10000.0, :vol 0.0, :r-vol 0.0, :sharpe 0.0, :r-sharpe 0.0, :pnl-pt 0.0, :max-drawdown 0.0}

In [17]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "volatility")
  data))

#'clojure-backtesting.demo/data-to-plot

In [18]:
(eval-report -1)


|      :date | :tot-value |    :vol |  :r-vol |  :sharpe | :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+----------+-----------+---------+---------------|
| 1980-12-16 |     $10000 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $0 |        0.0000 |
| 1980-12-17 |     $10016 | 0.0407% | 0.0407% |  1.7321% |   1.7321% |      $8 |      100.0000 |
| 1980-12-18 |     $10016 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $8 |        0.0000 |
| 1980-12-19 |     $10066 | 0.0598% | 0.0598% |  4.8019% |   4.8019% |     $22 |      100.0000 |
| 1980-12-22 |     $10066 | 0.0532% | 0.0532% |  5.3929% |   5.3929% |     $22 |      100.0000 |
| 1980-12-23 |     $10100 | 0.0499% | 0.0499% |  8.6792% |   8.6792% |     $25 |      100.0000 |
| 1980-12-24 |     $10100 | 0.0475% | 0.0475% |  9.1298% |   9.1298% |     $25 |      100.0000 |
| 1980-12-26 |     $10126 | 0.0477% | 0.0477% | 11.4442% |  11.4442% |     $25 |      100.0000 |
| 1980-12-29 |     $10126 | 0

nil

In [19]:
(plot data-to-plot :plot :date :vol true)

### 3. Plot sharpe ratio

In [20]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe true)

### 4. Plot stock price

In [21]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [22]:
(first data)

{:date "1980-12-17", :tic "AAPL", :price 25.9375, :aprc "26.09", :quantity 50}

In [23]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price true)